# SYNGENTA CROP CHALLENGE

# Machine Learning to predict crop yield

##### The challenge is to understand and classify crop yield based on two set of environmental features; heat and drought. We have modelled both separately.

## Importing and exploring data

In [1]:
import pandas as pd

performance_data = pd.read_csv('performance_data.csv')
submission_template = pd.read_csv('submission_template.csv')
weather_data = pd.read_csv('weather_data.csv')

In [2]:
performance_data.head()

HYBRID_ID ENV_ID  HYBRID_MG  ENV_MG     YIELD  YEAR   LAT  LONG  PLANT_DATE  \
0     H2782  Env_1          0       0  107.9577  2008  49.5 -98.0  2008-05-06   
1     H2782  Env_2          0       0   85.7498  2008  49.3 -98.1  2008-05-14   
2     H2240  Env_3          0       0   74.6116  2011  49.3 -98.0  2011-05-17   
3     H1527  Env_3          0       0   83.8191  2011  49.3 -98.0  2011-05-17   
4     H1369  Env_3          0       0   81.7917  2011  49.3 -98.0  2011-05-17   

  HARVEST_DATE  ...  ENV_YIELD_STD  ELEVATION  CLAY  SILT  SAND    AWC   PH  \
0   2008-11-03  ...       7.591866     870.65  22.7  23.0  54.5  18.65  7.2   
1   2008-10-22  ...       7.184953     942.41  22.7  22.0  55.8  18.75  7.3   
2   2011-10-17  ...       4.583234     903.46  22.8  21.5  55.8  18.95  7.4   
3   2011-10-17  ...       4.583234     903.46  22.8  21.5  55.8  18.95  7.4   
4   2011-10-17  ...       4.583234     903.46  22.8  21.5  55.8  18.95  7.4   

    OM   CEC  KSAT  
0  6.1  24.2   9.4  
1  6.9  25.2  10.2  
2  6.7  25.5   9.9  
3  6.7  25.5   9.9  
4  6.7  25.5   9.9  

[5 rows x 22 columns]

## Transforming weather data into useful metrics



Environmental data will be grouped by ENV_ID. Both averages and standard deviations will be obtained so as to capture the differences in weather. NB: Same average temperatures can hide different extreme environments. 

In [3]:
#Group data by crop id
weather_data_grouped_mean = weather_data.groupby(by='ENV_ID').mean()
weather_data_grouped_std = weather_data.groupby(by='ENV_ID').std()
weather_data_grouped = weather_data_grouped_mean.join(weather_data_grouped_std,
                                                      lsuffix='_AVG',
                                                     rsuffix='_STD')
weather_data_grouped = weather_data_grouped.drop('DAY_NUM_AVG',axis=1)
weather_data_grouped.head()

DAYL_AVG  PREC_AVG    SRAD_AVG    SWE_AVG   TMAX_AVG  TMIN_AVG  \
ENV_ID                                                                         
Env_1     43200.000043  1.684932  276.111781  11.167123   8.093151 -3.256164   
Env_10    43200.946768  3.079452  294.978630  19.868493  12.095890  2.241096   
Env_100   43200.000070  2.901370  296.714521  20.679452  12.616438  0.632877   
Env_1000  43200.946854  2.000000  352.368219   0.997260  18.323288  2.883562   
Env_1001  43200.946854  2.561644  341.681097   2.443836  17.701370  2.924658   

              VP_AVG  DAY_NUM_STD      DAYL_STD  PREC_STD    SRAD_STD  \
ENV_ID                                                                  
Env_1     630.356164   105.510663  10075.200862  4.544882  131.085275   
Env_10    874.191781   105.510663   8183.382184  5.974924  134.977729   
Env_100   820.054795   105.510663   8795.413435  7.152662  119.593751   
Env_1000  824.876712   105.510663   6955.702533  6.970897  111.010912   
Env_1001  862.246575   105.510663   6955.702533  8.697269  112.386846   

            SWE_STD   TMAX_STD   TMIN_STD      VP_STD  
ENV_ID                                                 
Env_1     16.046875  14.926959  13.574307  535.634711  
Env_10    31.810984  11.625704   9.839269  559.526111  
Env_100   30.413609  12.877466  12.410814  614.102577  
Env_1000   2.447805  11.374718  10.219944  674.104452  
Env_1001   5.729262  11.039508  10.119774  629.404388

In [4]:
## Joining performance and weather data
joined_df = performance_data.join(weather_data,on='ENV_ID')
joined_df_grouped = performance_data.join(weather_data_grouped,on='ENV_ID')
joined_df_grouped.columns

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [ ]:
# Discerning between heat and drought varibles
heat_stress_df = joined_df_grouped[['HYBRID_ID','ENV_ID','TMAX_AVG','TMAX_STD',
                            'TMIN_AVG','TMIN_STD','DAYL_AVG','DAYL_STD',
                           'SRAD_AVG','SRAD_STD','YIELD']]
drought_stress_df = joined_df_grouped[['HYBRID_ID','ENV_ID','IRRIGATION','PREC_AVG','PREC_STD','KSAT',
                              'SWE_AVG','SWE_STD','VP_AVG','VP_STD','AWC','YIELD']]

In [ ]:
joined_df_grouped.head()

## Binning function

In [ ]:
factor_df = joined_df[['LONG', 'LAT', 'PREC', 'TMAX', 'TMIN']].drop_duplicates()
#determine min, mean, max
#segment factor from 1 to 4

def factor_seg(df, factor):
    min_ = factor_df[factor].min()
    max_ = factor_df[factor].max()
    mean_ = factor_df[factor].mean()
    #binning
    diff = (max_ - min_)/4
    b_l = list()
    for x in range(len(df)):
        temp = 0
        if df.iloc[x][factor] < (min_ + diff):
            temp = 1
        elif df.iloc[x][factor] < (min_ + 2*diff):
            temp = 2
        elif df.iloc[x][factor] < (min_ + 3*diff):
            temp = 3
        else:
            temp = 4
        b_l.append(temp)
    df.insert(loc=0, column=factor + '_binning', value=b_l)
    return df
test_df = factor_seg(factor_df, 'PREC')
#test_df
#test_df.loc[:, 'PREC_binning']

## Folium map with popup markers for every coordinate pair

In [ ]:
#folium map
#For every coordinate pair: a popup shows values for 'PREC', 'IRR', 'ELEVATION', 'TMAX', 'TMIN'
import folium
m = folium.Map(location=[39, -96], zoom_start=5, tiles='Stamen Terrain')
folium_data = joined_df[['LONG', 'LAT', 'PREC','IRRIGATION', 'ELEVATION', 'TMAX', 'TMIN']].drop_duplicates()
folium
for x in range(len(folium_data)):
    folium.Marker([folium_data.iloc[x]['LAT'], folium_data.iloc[x]['LONG']], popup=(f"""Prec: {folium_data.iloc[x]['PREC']:.2f} IRR: {folium_data.iloc[x]['IRRIGATION']} Elev: {folium_data.iloc[x]['ELEVATION']:.0f} Tmax: {folium_data.iloc[x]['TMAX']:.2f} Tmin: {folium_data.iloc[x]['TMIN']:.2f}""")).add_to(m)
m


## Correlation heatmap
There are no strong correlations between weather conditions and yield if we look at the aggregated data. We can conclude we'll have to go one level deeper and do the analysis on every specific Hybrid_ID

In [ ]:
# Import modules
import seaborn as sns
import numpy as np

# Prepare dataframe to redo correlation matrix
joined_df_drop = joined_df.drop(['HYBRID_ID','ENV_ID','ENV_YIELD_MEAN','ENV_YIELD_STD'],axis='columns')
corr_matrix_joined = joined_df_drop.corr()

# Draw heatmap
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,10))
sns.heatmap(corr_matrix_joined,cmap='BuGn')
top_5 = corr_matrix_joined['YIELD'].sort_values(ascending=False).iloc[1:6]
bottom_5 = corr_matrix_joined['YIELD'].sort_values(ascending=True).iloc[0:5]
print(top_5,bottom_5)

# Heat stress variables

In [ ]:
# Heat stress variables to yield correlation
# Import modules
import seaborn as sns
import numpy as np

# Prepare dataframe to redo correlation matrix
heat_stress_df_drop = heat_stress_df.drop(['HYBRID_ID','ENV_ID','ENV_YIELD_MEAN','ENV_YIELD_STD'],axis='columns')
corr_matrix_heat = heat_stress_df_drop.corr()

# Draw heatmap
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,10))
sns.heatmap(corr_matrix_heat,cmap='YlOrRd')

print(corr_matrix_heat['YIELD'].sort_values(ascending=False))
#corr_matrix_heat['YIELD'].sort_values(ascending=True)

In [ ]:
heat_stress_features_list = ['TMAX_AVG','DAYL_AVG','SRAD_AVG', 'TMAX_STD', 'DAYL_STD','SRAD_STD']

#heat_stress_df_drop = df[heat_stress_features_list]
#heat_stress_features = [x for x in heat_stress_features_list]

fig, ax = plt.subplots(round(len(heat_stress_features_list) / 3), 3, figsize = (18, 12))

for i, ax in enumerate(fig.axes):
    if i < len(heat_stress_features_list) - 1:
        sns.regplot(x=heat_stress_features_list[i],y='YIELD', data=heat_stress_df_drop, ax=ax)

# Drought stress variables

In [ ]:
# Drought stress variables to yield correlation
# Import modules
import seaborn as sns
import numpy as np

# Prepare dataframe to redo correlation matrix
drought_stress_df_drop = drought_stress_df.drop(['HYBRID_ID','ENV_ID','ENV_YIELD_MEAN','ENV_YIELD_STD'],axis='columns')
corr_matrix_drought = drought_stress_df_drop.corr()

# Draw heatmap
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(12,10))
sns.heatmap(corr_matrix_heat,cmap='YlGnBu')

print(corr_matrix_drought['YIELD'].sort_values(ascending=False))
#corr_matrix_drought['YIELD'].sort_values(ascending=True)

In [ ]:
# Correlation between each drought variable and yield
drought_stress_features_list = ['PREC_STD','VP_STD','AWC', 'PREC_AVG','VP_AVG','KSAT','SWE_STD','SWE_AVG']

fig, ax = plt.subplots(round(len(drought_stress_features_list) / 3), 3, figsize = (18, 12))

for i, ax in enumerate(fig.axes):
    if i < len(drought_stress_features_list) - 1:
        sns.regplot(x=drought_stress_features_list[i],y='YIELD', data=drought_stress_df_drop, ax=ax)

# Exploring categorical variables
- Captures non-numeric variable not featured in the correlation matrices

In [ ]:
# Ordered from least to most irrigated 
var = 'IRRIGATION'
data = pd.concat([joined_df['YIELD'], joined_df[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="YIELD", data=data, order=["NONE", "DRY", "ECO","LIRR","IRR"])
fig.axis(ymin=0, ymax=200);

In [ ]:
# same for PREC
var = 'IRRIGATION'
data = pd.concat([joined_df['PREC_AVG'], joined_df[var]], axis=1)
f, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="PREC_AVG", data=data, order=["NONE", "DRY", "ECO","LIRR","IRR"])
fig.axis(ymin=0, ymax=10);

# HEAT STRESS

In [ ]:
#Setting non-numeric variables to index
heat_stress_df = heat_stress_df.set_index(['HYBRID_ID','ENV_ID'])
heat_stress_df.head()

In [ ]:
print('Total performance observations: ' + str(len(heat_stress_df)))
print('Total unique hybrids: ' + str(len(heat_stress_df.reset_index()['HYBRID_ID'].unique())))


## Setting up a Random Forest model

In [ ]:
# Splitting data into training (70%), testing (20%) and validation (10%)
import sklearn
from sklearn.model_selection import train_test_split

train, test = train_test_split(heat_stress_df, test_size = 0.1, random_state=1)
x_train = train.iloc[:,0:-1]
y_train = train['YIELD']
x_test = test.iloc[:,0:-1]
y_test = test['YIELD']


In [ ]:
# Setting up model
import numpy as np
from sklearn import linear_model
from sklearn import tree

tree_model_hs = tree.DecisionTreeRegressor()

# Fitting models
tree_model_hs.fit(x_train,y_train)


## Model score

In [ ]:
#Get score on the testing data
print('Model score equals: %.3f' % tree_model_hs.score(x_test,y_test))


## Understanding the underlying parameters 

A key parameter in the Decision Tree Regressor is the minimum number of instances, i.e. the minimum number of samples required to be at a leaf node. We would expect the model to reduce its MSE to the training data when we decrease the minimum samples required to be a leaf node because more nodes would be added and the data would be more "fit" to the training data. However, with such procedure we would be exposed to an overfitting risk of the model to the training data and hence our MSE to the testing data would increase. 

Surprisingly, the figure below shows this does not apply in our case. The lower the number of instances required in a leaf node, the better for both the training and testing data. We can conclude that having a "super-tree" with as much leafes as number of instances is the best approach for both the training and testing data.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(16,8))
ax0 = fig.add_subplot(111) 
RMSE_train = []
RMSE_test = []
iterations = 100

for i in range(1,iterations):
    #Parametrize the model and let i be the number of minimum instances per leaf node
    regression_model = tree.DecisionTreeRegressor(criterion="mse",min_samples_leaf=i)   
    #Train the model
    regression_model.fit(x_train,y_train)
    #Predict query instances
    predicted_train = regression_model.predict(x_train)
    predicted_test = regression_model.predict(x_test)
    #Calculate and append the RMSEs
    RMSE_train.append(np.sqrt(np.sum(((y_train-predicted_train)**2)/len(y_train))))
    RMSE_test.append(np.sqrt(np.sum(((y_test-predicted_test)**2)/len(y_test))))
    
ax0.plot(range(1,iterations),RMSE_test,label='Test_Data')
ax0.plot(range(1,iterations),RMSE_train,label='Train_Data')
ax0.legend()
ax0.set_title('RMSE with respect to the minumim number of instances per node')
ax0.set_xlabel('#Instances')
ax0.set_ylabel('RMSE')
plt.show()

The same perspective but with model scores (ranging from 0 to 1).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(16,8))
ax0 = fig.add_subplot(111) 
SCORE_train = []
SCORE_test = []
iterations = 100

for i in range(1,iterations):
    #Parametrize the model and let i be the number of minimum instances per leaf node
    regression_model = tree.DecisionTreeRegressor(criterion="mse",min_samples_leaf=i)   
    #Train the model
    regression_model.fit(x_train,y_train)
    #Get scores
    SCORE_train.append(regression_model.score(x_train,y_train))
    SCORE_test.append(regression_model.score(x_test,y_test))
        
ax0.plot(range(1,iterations),SCORE_test,label='Test_Data')
ax0.plot(range(1,iterations),SCORE_train,label='Train_Data')
ax0.legend()
ax0.set_title('SCORE with respect to the minumim number of instances per node')
ax0.set_xlabel('#Instances')
ax0.set_ylabel('SCORE')
plt.show()


## Decision Tree illustration


DAYL_STD is the most decisive factor when it comes to predicting yield


In [ ]:
import pydotplus 
from IPython.display import Image

feature_names = [key for key in heat_stress_df.columns if not key == 'YIELD']

dot_data = tree.export_graphviz(tree_model_hs,
                                max_depth=2,
                                out_file=None,
                                feature_names=feature_names)

graph = pydotplus.graphviz.graph_from_dot_data(dot_data)

Image(graph.create_png())

## Feature importances and individual scores


In [ ]:
# Discerning feature importances
tree_model_hs.feature_importances_

# Normalizing dataframe
norm_hs = sklearn.preprocessing.normalize(heat_stress_df,axis=0)
df_hs = pd.DataFrame(norm_hs)
df_hs.describe()
heat_stress_df.describe()
norm_hs

# DROUGHT STRESS


In [ ]:
#Setting non-numeric variables to index
drought_stress_df = drought_stress_df.set_index(['HYBRID_ID','ENV_ID'])
drought_stress_df.head()


### Cleaning data


In [ ]:
# Converting irrigation to numeric values
def irrigation_converter(string):
    import numpy as np
    if string == np.nan:
        return np.nan
    if string == 'NONE' or string == 'DRY':
        return 0
    if string == 'ECO':
        return 1
    if string == 'LIRR':
        return 2
    if string == 'IRR':
        return 3
    else:
        next

drought_stress_df['IRRIGATION'] = drought_stress_df['IRRIGATION'].apply(irrigation_converter)
print('Total performance observations: ' + str(len(drought_stress_df)))
print('Total unique hybrids: ' + str(len(drought_stress_df.reset_index()['HYBRID_ID'].unique())))


In [ ]:
# Drop nan
drought_stress_df.dropna(inplace=True)

print('Total performance observations: ' + str(len(drought_stress_df)))
print('Total unique hybrids: ' + str(len(drought_stress_df.reset_index()['HYBRID_ID'].unique())))



## Setting up a Random Forest model


In [ ]:
# Splitting data
import sklearn
from sklearn.model_selection import train_test_split

train, test = train_test_split(drought_stress_df, test_size = 0.3, random_state=1)
x_train = train.iloc[:,0:-1]
y_train = train['YIELD']
x_test = test.iloc[:,0:-1]
y_test = test['YIELD']

In [ ]:
# Setting up model
import numpy as np
from sklearn import linear_model
from sklearn import tree

tree_model_ds = tree.DecisionTreeRegressor()

# Fitting models
tree_model_ds.fit(x_train,y_train)

## Model score

In [ ]:
#Get score on the testing data
print(tree_model_ds.score(x_test,y_test))

## Decision Tree illustration


IRRIGATION is the most decisive factor when it comes to predicting yield

In [ ]:
import pydotplus 
from IPython.display import Image

feature_names = [key for key in drought_stress_df.columns if not key == 'YIELD']

dot_data = tree.export_graphviz(tree_model_ds,
                                max_depth=2,
                                out_file=None,
                                feature_names=feature_names)

graph = pydotplus.graphviz.graph_from_dot_data(dot_data)

Image(graph.create_png())


## Find feature importance